In [47]:
import os
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline
print ("plt version:[{}]".format(matplotlib.__version__))
print ("gym version:[{}]".format(gym.__version__))

plt version:[3.4.3]
gym version:[0.20.0]


dataset :https://people.sc.fsu.edu/~jburkardt/datasets/tsp/tsp.html

In [24]:
tsp_data_txt = open('tsp_dataset.txt')

In [25]:
lines = tsp_data_txt.readlines()

In [31]:
data = {}

distance_matrix = np.zeros((len(lines),len(lines)))
for i,line in enumerate(lines):
    for j,element in enumerate((line.split('\n')[0].split('       '))):
        if(element == ''):
            continue
        
        distance_matrix[i][j-1] = int(element.strip())
data['distance_matrix']= distance_matrix
data['num_vehicles'] = 1
data['depot'] = 0


In [41]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [42]:
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [43]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [44]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [45]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

<h1>TSP OR solutions</h1>

In [46]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 291 miles
Route for vehicle 0:
 0 -> 12 -> 1 -> 14 -> 8 -> 4 -> 6 -> 2 -> 11 -> 13 -> 9 -> 7 -> 5 -> 3 -> 10 -> 0



In [48]:
15*15

225